In [51]:
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.nn as nn

import os.path
import random
from random import uniform
import numpy as np
import torch
import torch.utils.data as data
import torchvision.transforms as transforms
import cv2
import matplotlib.pyplot as plt
import argparse
import math
os.environ['CUDA_LAUNCH_BLOCKING']='1'

In [ ]:
## 好神秘

## Data Augmentation

In [52]:
class Compose(object):

    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, data):
        for function_ in self.transforms:
            data = function_(data)
        return data


class Crop(object):

    def __init__(self, max_crop=0.1):
        super().__init__()
        self.max_crop = max_crop

    def __call__(self, data):
        image, label = data
        height, width = image.shape[:2]
        xmin = width
        ymin = height
        xmax = 0
        ymax = 0
        for lb in label:
            xmin = min(xmin, lb[0])
            ymin = min(ymin, lb[1])
            xmax = max(xmax, lb[2])
            ymax = max(ymax, lb[2])
        cropped_left = uniform(0, self.max_crop)
        cropped_right = uniform(0, self.max_crop)
        cropped_top = uniform(0, self.max_crop)
        cropped_bottom = uniform(0, self.max_crop)
        new_xmin = int(min(cropped_left * width, xmin))
        new_ymin = int(min(cropped_top * height, ymin))
        new_xmax = int(max(width - 1 - cropped_right * width, xmax))
        new_ymax = int(max(height - 1 - cropped_bottom * height, ymax))

        image = image[new_ymin:new_ymax, new_xmin:new_xmax, :]
        label = [[lb[0] - new_xmin, lb[1] - new_ymin, lb[2] - new_xmin, lb[3] - new_ymin, lb[4]] for lb in label]

        return image, label


class VerticalFlip(object):

    def __init__(self, prob=0.5):
        super().__init__()
        self.prob = prob

    def __call__(self, data):
        image, label = data
        if uniform(0, 1) >= self.prob:
            image = cv2.flip(image, 1)
            width = image.shape[1]
            label = [[width - lb[2], lb[1], width - lb[0], lb[3], lb[4]] for lb in label]
        return image, label


class HSVAdjust(object):

    def __init__(self, hue=30, saturation=1.5, value=1.5, prob=0.5):
        super().__init__()
        self.hue = hue
        self.saturation = saturation
        self.value = value
        self.prob = prob

    def __call__(self, data):

        def clip_hue(hue_channel):
            hue_channel[hue_channel >= 360] -= 360
            hue_channel[hue_channel < 0] += 360
            return hue_channel

        image, label = data
        adjust_hue = uniform(-self.hue, self.hue)
        adjust_saturation = uniform(1, self.saturation)
        if uniform(0, 1) >= self.prob:
            adjust_saturation = 1 / adjust_saturation
        adjust_value = uniform(1, self.value)
        if uniform(0, 1) >= self.prob:
            adjust_value = 1 / adjust_value
        image = image.astype(np.float32) / 255
        image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        image[:, :, 0] += adjust_hue
        image[:, :, 0] = clip_hue(image[:, :, 0])
        image[:, :, 1] = np.clip(adjust_saturation * image[:, :, 1], 0.0, 1.0)
        image[:, :, 2] = np.clip(adjust_value * image[:, :, 2], 0.0, 1.0)

        image = cv2.cvtColor(image, cv2.COLOR_HSV2RGB)
        image = (image * 255).astype(np.float32)

        return image, label


class Resize(object):

    def __init__(self, image_size):
        super().__init__()
        self.image_size = image_size

    def __call__(self, data):
        image, label = data
        height, width = image.shape[:2]
        image = cv2.resize(image, (self.image_size, self.image_size))
        width_ratio = float(self.image_size) / width
        height_ratio = float(self.image_size) / height
        new_label = []
        for lb in label:
            resized_xmin = lb[0] * width_ratio
            resized_ymin = lb[1] * height_ratio
            resized_xmax = lb[2] * width_ratio
            resized_ymax = lb[3] * height_ratio
            resize_width = resized_xmax - resized_xmin
            resize_height = resized_ymax - resized_ymin
            new_label.append([resized_xmin, resized_ymin, resize_width, resize_height, lb[4]])

        return image, new_label

## Yolo dataset

In [53]:
class yoloDataset(data.Dataset):
    image_size = 416 # Size to be aligned

    # Parsing data list
    def __init__(self,root,list_file,train):
        self.root = root
        self.train = train
        self.fnames = []
        self.objects = [] # [x_min, y_min, x_max, y_max, class]
        self.classes = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow',
                        'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train',
                        'tvmonitor']
        self.num_classes = len(self.classes)

        with open(list_file) as f:
            lines  = f.readlines()

        # format of each line: filename (x_min, y_min, x_max, y_max, label) * object_num
        for line in lines:
            splited = line.strip().split() # .strip(): reomove space, tab from the end of each line
            self.fnames.append(splited[0])
            num_boxes = (len(splited) - 1) // 5
            one_object = []
            for i in range(num_boxes):
                x_min = float(splited[1+5*i])
                y_min = float(splited[2+5*i])
                x_max = float(splited[3+5*i])
                y_max = float(splited[4+5*i])
                c = splited[5+5*i]
                # int(c)+1
                one_object.append([x_min,y_min,x_max,y_max, int(c)])
            self.objects.append(one_object)
        self.num_samples = len(self.objects)

    # Getting single transformed, preprocessed image and its bounding boxes
    def __getitem__(self,idx):
        fname = self.fnames[idx]
        img = cv2.imread(os.path.join(self.root+fname))
        image_objects = self.objects[idx]
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if self.train:
            transformations = Compose([HSVAdjust(), VerticalFlip(), Crop(), Resize(self.image_size)])
        else:
            transformations = Compose([Resize(self.image_size)])

        # h,w,_ = img.shape
        # boxes /= torch.Tensor([w,h,w,h]).expand_as(boxes)    -> replaced by batch-norm(?)
        # img = self.subMean(img,self.mean)
        # to see the whih grid is the box is located at
        # .expand_as(other): expand this tensor as other
        # [w, h, w, h] (1, 4) will be expanded to (#box, 4)
        img, image_objects = transformations((img, image_objects))
#         print(len(image_objects))

        # img.shape (416,416,3) -> (3,416,416)
        return np.transpose(np.array(img, dtype=np.float32), (2, 0, 1)), np.array(image_objects, dtype=np.float32)

    def __len__(self):
        return self.num_samples

## Yolo structure

In [54]:
def build_layer(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False, isMaxpool=False):
    if(isMaxpool):
        return nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding,bias=bias),
                             nn.BatchNorm2d(out_channels),
                             nn.LeakyReLU(0.1, inplace=True),
                             nn.MaxPool2d(2,2))
    else:
        return nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding,bias=bias),
                             nn.BatchNorm2d(out_channels),
                             nn.LeakyReLU(0.1, inplace=True))

In [56]:
class Yolov2(nn.Module):
    def __init__(self, num_classes, 
                 anchors = [(1.3221, 1.73145), (3.19275, 4.00944), (5.05587, 8.09892), (9.47112, 4.84053),(11.2364, 10.0071)]):
        super(Yolov2,self).__init__()
        self.num_classes = num_classes
        self.anchors = anchors
        # darnet19
        self.stage1_c1 = build_layer(3, 32, 3, 1, 1, bias=False, isMaxpool=True)        
        self.stage1_c2 = build_layer(32, 64, 3, 1, 1, bias=False, isMaxpool=True)
        
        self.stage1_c3 = build_layer(64, 128, 3, 1, 1, bias=False, isMaxpool=False)        
        self.stage1_c4 = build_layer(128, 64, 1, 1, 0, bias=False, isMaxpool=False)        
        self.stage1_c5 = build_layer(64, 128, 3, 1, 1, bias=False, isMaxpool=True)  
        
        self.stage1_c6 = build_layer(128, 256, 3, 1, 1, bias=False, isMaxpool=False)        
        self.stage1_c7 = build_layer(256, 128, 1, 1, 0, bias=False, isMaxpool=False)        
        self.stage1_c8 = build_layer(128, 256, 3, 1, 1, bias=False, isMaxpool=True) 
        
        self.stage1_c9 = build_layer(256, 512, 3, 1, 1, bias=False, isMaxpool=False)        
        self.stage1_c10 = build_layer(512, 256, 1, 1, 0, bias=False, isMaxpool=False)        
        self.stage1_c11 = build_layer(256, 512, 3, 1, 1, bias=False, isMaxpool=False)        
        self.stage1_c12 = build_layer(512, 256, 1, 1, 0, bias=False, isMaxpool=False)        
        self.stage1_c13 = build_layer(256, 512, 3, 1, 1, bias=False, isMaxpool=False)
        
        self.stage2_maxpool = nn.MaxPool2d(2, 2)
        self.stage2_c1 = build_layer(512, 1024, 3, 1, 1, bias=False, isMaxpool=False)        
        self.stage2_c2 = build_layer(1024, 512, 1, 1, 0, bias=False, isMaxpool=False)        
        self.stage2_c3 = build_layer(512, 1024, 3, 1, 1, bias=False, isMaxpool=False)        
        self.stage2_c4 = build_layer(1024, 512, 1, 1, 0, bias=False, isMaxpool=False)        
        self.stage2_c5 = build_layer(512, 1024, 3, 1, 1, bias=False, isMaxpool=False)
        
        self.stage2_c6 = build_layer(1024, 1024, 3, 1, 1, bias=False, isMaxpool=False)
        self.stage2_c7 = build_layer(1024, 1024, 3, 1, 1, bias=False, isMaxpool=False)
        
        self.stage2_residual_c1 = build_layer(512, 64, 1, 1, 0, bias=False, isMaxpool=False)
        
        # classify head
        self.stage3_c1 = build_layer(256+1024, 1024, 1, 1, 0, bias=False, isMaxpool=False)
        self.stage3_c2 = nn.Conv2d(1024, len(self.anchors)*(5+self.num_classes), 1, 1, 0, bias=False)
#         self.avgpool = nn.AvgPool2d((1, 1))
#         self.softmax = nn.Softmax(dim=1)
    
    def forward(self, input):
        output_1 = self.stage1_c1(input)
        output_1 = self.stage1_c2(output_1)
        output_1 = self.stage1_c3(output_1)
        output_1 = self.stage1_c4(output_1)
        output_1 = self.stage1_c5(output_1)
        output_1 = self.stage1_c6(output_1)
        output_1 = self.stage1_c7(output_1)
        output_1 = self.stage1_c8(output_1)
        output_1 = self.stage1_c9(output_1)
        output_1 = self.stage1_c10(output_1)
        output_1 = self.stage1_c11(output_1)
        output_1 = self.stage1_c12(output_1)
        output_1 = self.stage1_c13(output_1)

        residual = output_1

        output_2 = self.stage2_maxpool(output_1)
        output_2 = self.stage2_c1(output_2)
        output_2 = self.stage2_c2(output_2)
        output_2 = self.stage2_c3(output_2)
        output_2 = self.stage2_c4(output_2)
        output_2 = self.stage2_c5(output_2)
        output_2 = self.stage2_c6(output_2)
        output_2 = self.stage2_c7(output_2)
        
        # rearrange
        output_residual = self.stage2_residual_c1(residual)
        batch_size, num_channel, height, width = output_residual.data.size()
        output_residual = output_residual.view(batch_size, int(num_channel / 4), height, 2, width, 2).contiguous()
        output_residual = output_residual.permute(0, 3, 5, 1, 2, 4).contiguous()
        output_residual = output_residual.view(batch_size, -1, int(height / 2), int(width / 2))
        
        # concate residual block
        output = torch.cat((output_2, output_residual), 1)
        output = self.stage3_c1(output)
        output = self.stage3_c2(output)
#         output = self.avgpool(output)
#         output = self.softmax(output)
        
        return output

In [57]:
from torchsummary import summary
model = Yolov2(20).cuda()
summary(model, (3,416,416))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 416, 416]             864
       BatchNorm2d-2         [-1, 32, 416, 416]              64
         LeakyReLU-3         [-1, 32, 416, 416]               0
         MaxPool2d-4         [-1, 32, 208, 208]               0
            Conv2d-5         [-1, 64, 208, 208]          18,432
       BatchNorm2d-6         [-1, 64, 208, 208]             128
         LeakyReLU-7         [-1, 64, 208, 208]               0
         MaxPool2d-8         [-1, 64, 104, 104]               0
            Conv2d-9        [-1, 128, 104, 104]          73,728
      BatchNorm2d-10        [-1, 128, 104, 104]             256
        LeakyReLU-11        [-1, 128, 104, 104]               0
           Conv2d-12         [-1, 64, 104, 104]           8,192
      BatchNorm2d-13         [-1, 64, 104, 104]             128
        LeakyReLU-14         [-1, 64, 1

## Loss function

In [58]:
class YoloLoss(nn.modules.loss._Loss):
    # The loss I borrow from LightNet repo.
    def __init__(self, num_classes, anchors, reduction=32, coord_scale=1.0, noobject_scale=1.0,
                 object_scale=5.0, class_scale=1.0, thresh=0.6):
        super(YoloLoss, self).__init__()
        self.num_classes = num_classes
        self.num_anchors = len(anchors)
        self.anchor_step = len(anchors[0])
        self.anchors = torch.Tensor(anchors)
        self.reduction = reduction

        self.coord_scale = coord_scale
        self.noobject_scale = noobject_scale
        self.object_scale = object_scale
        self.class_scale = class_scale
        self.thresh = thresh

    def forward(self, output, target):

        batch_size = output.data.size(0)
        height = output.data.size(2)
        width = output.data.size(3)

        # Get x,y,w,h,conf,cls
        output = output.view(batch_size, self.num_anchors, -1, height * width)
        coord = torch.zeros_like(output[:, :, :4, :])
        coord[:, :, :2, :] = output[:, :, :2, :].sigmoid()  
        coord[:, :, 2:4, :] = output[:, :, 2:4, :]
        conf = output[:, :, 4, :].sigmoid()
        cls = output[:, :, 5:, :].contiguous().view(batch_size * self.num_anchors, self.num_classes,
                                                    height * width).transpose(1, 2).contiguous().view(-1,
                                                                                                      self.num_classes)

        # Create prediction boxes
        pred_boxes = torch.FloatTensor(batch_size * self.num_anchors * height * width, 4)
        lin_x = torch.range(0, width - 1).repeat(height, 1).view(height * width)
        lin_y = torch.range(0, height - 1).repeat(width, 1).t().contiguous().view(height * width)
        anchor_w = self.anchors[:, 0].contiguous().view(self.num_anchors, 1)
        anchor_h = self.anchors[:, 1].contiguous().view(self.num_anchors, 1)

        if torch.cuda.is_available():
            pred_boxes = pred_boxes.cuda()
            lin_x = lin_x.cuda()
            lin_y = lin_y.cuda()
            anchor_w = anchor_w.cuda()
            anchor_h = anchor_h.cuda()

        pred_boxes[:, 0] = (coord[:, :, 0].detach() + lin_x).view(-1)
        pred_boxes[:, 1] = (coord[:, :, 1].detach() + lin_y).view(-1)
        pred_boxes[:, 2] = (coord[:, :, 2].detach().exp() * anchor_w).view(-1)
        pred_boxes[:, 3] = (coord[:, :, 3].detach().exp() * anchor_h).view(-1)
        pred_boxes = pred_boxes.cpu()

        # Get target values
        coord_mask, conf_mask, cls_mask, tcoord, tconf, tcls = self.build_targets(pred_boxes, target, height, width)
        coord_mask = coord_mask.expand_as(tcoord)
        tcls = tcls[cls_mask].view(-1).long()
        cls_mask = cls_mask.view(-1, 1).repeat(1, self.num_classes)

        if torch.cuda.is_available():
            tcoord = tcoord.cuda()
            tconf = tconf.cuda()
            coord_mask = coord_mask.cuda()
            conf_mask = conf_mask.cuda()
            tcls = tcls.cuda()
            cls_mask = cls_mask.cuda()

        conf_mask = conf_mask.sqrt()
        cls = cls[cls_mask].view(-1, self.num_classes)

        # Compute losses
        mse = nn.MSELoss(size_average=False)
        ce = nn.CrossEntropyLoss(size_average=False)
        self.loss_coord = self.coord_scale * mse(coord * coord_mask, tcoord * coord_mask) / batch_size
        self.loss_conf = mse(conf * conf_mask, tconf * conf_mask) / batch_size
        self.loss_cls = self.class_scale * 2 * ce(cls, tcls) / batch_size
        self.loss_tot = self.loss_coord + self.loss_conf + self.loss_cls

        return self.loss_tot, self.loss_coord, self.loss_conf, self.loss_cls

    def build_targets(self, pred_boxes, ground_truth, height, width):
        batch_size = len(ground_truth)

        conf_mask = torch.ones(batch_size, self.num_anchors, height * width, requires_grad=False) * self.noobject_scale
        coord_mask = torch.zeros(batch_size, self.num_anchors, 1, height * width, requires_grad=False)
        cls_mask = torch.zeros(batch_size, self.num_anchors, height * width, requires_grad=False).bool()
        tcoord = torch.zeros(batch_size, self.num_anchors, 4, height * width, requires_grad=False)
        tconf = torch.zeros(batch_size, self.num_anchors, height * width, requires_grad=False)
        tcls = torch.zeros(batch_size, self.num_anchors, height * width, requires_grad=False)

        for b in range(batch_size):
            if len(ground_truth[b]) == 0:
                continue

            # Build up tensors
            cur_pred_boxes = pred_boxes[
                             b * (self.num_anchors * height * width):(b + 1) * (self.num_anchors * height * width)]
            if self.anchor_step == 4:
                anchors = self.anchors.clone()
                anchors[:, :2] = 0
            else:
                anchors = torch.cat([torch.zeros_like(self.anchors), self.anchors], 1)
            gt = torch.zeros(len(ground_truth[b]), 4)
            for i, anno in enumerate(ground_truth[b]):
                gt[i, 0] = (anno[0] + anno[2] / 2) / self.reduction
                gt[i, 1] = (anno[1] + anno[3] / 2) / self.reduction
                gt[i, 2] = anno[2] / self.reduction
                gt[i, 3] = anno[3] / self.reduction

            # Set confidence mask of matching detections to 0
            iou_gt_pred = bbox_ious(gt, cur_pred_boxes)
            mask = (iou_gt_pred > self.thresh).sum(0) >= 1
            conf_mask[b][mask.view_as(conf_mask[b])] = 0

            # Find best anchor for each ground truth
            gt_wh = gt.clone()
            gt_wh[:, :2] = 0
            iou_gt_anchors = bbox_ious(gt_wh, anchors)
            _, best_anchors = iou_gt_anchors.max(1)

            # Set masks and target values for each ground truth
            for i, anno in enumerate(ground_truth[b]):
                gi = min(width - 1, max(0, int(gt[i, 0])))
                gj = min(height - 1, max(0, int(gt[i, 1])))
                best_n = best_anchors[i]
                iou = iou_gt_pred[i][best_n * height * width + gj * width + gi]
                coord_mask[b][best_n][0][gj * width + gi] = 1
                cls_mask[b][best_n][gj * width + gi] = 1
                conf_mask[b][best_n][gj * width + gi] = self.object_scale
                tcoord[b][best_n][0][gj * width + gi] = gt[i, 0] - gi
                tcoord[b][best_n][1][gj * width + gi] = gt[i, 1] - gj
                tcoord[b][best_n][2][gj * width + gi] = math.log(max(gt[i, 2], 1.0) / self.anchors[best_n, 0])
                tcoord[b][best_n][3][gj * width + gi] = math.log(max(gt[i, 3], 1.0) / self.anchors[best_n, 1])
                tconf[b][best_n][gj * width + gi] = iou
                tcls[b][best_n][gj * width + gi] = int(anno[4])

        return coord_mask, conf_mask, cls_mask, tcoord, tconf, tcls


def bbox_ious(boxes1, boxes2):
    b1x1, b1y1 = (boxes1[:, :2] - (boxes1[:, 2:4] / 2)).split(1, 1)
    b1x2, b1y2 = (boxes1[:, :2] + (boxes1[:, 2:4] / 2)).split(1, 1)
    b2x1, b2y1 = (boxes2[:, :2] - (boxes2[:, 2:4] / 2)).split(1, 1)
    b2x2, b2y2 = (boxes2[:, :2] + (boxes2[:, 2:4] / 2)).split(1, 1)

    dx = (b1x2.min(b2x2.t()) - b1x1.max(b2x1.t())).clamp(min=0)
    dy = (b1y2.min(b2y2.t()) - b1y1.max(b2y1.t())).clamp(min=0)
    intersections = dx * dy

    areas1 = (b1x2 - b1x1) * (b1y2 - b1y1)
    areas2 = (b2x2 - b2x1) * (b2y2 - b2y1)
    unions = (areas1 + areas2.t()) - intersections

    return intersections / unions

In [59]:
def custom_collate_fn(batch):
    items = list(zip(*batch))
    items[0] = default_collate(items[0])
    items[1] = list(items[1])
    return items

## Train

In [60]:
batch_size = 8
root = 'VOCdevkit/VOC2007/JPEGImages/'
reduction = 32
momentum = 0.9
decay = 0.001
valid_interval = 10
epoches = 100
epoch_patience = 20
pre_trained_model_path = 'yolov2.weights'
es_min_delta = 0.0

def train():
    # check cuda
    if torch.cuda.is_available():
        torch.cuda.manual_seed(123)
    else:
        torch.manual_seed(123)
        
    # set scheduler
    learning_rate_schedule = {"0": 1e-5, "5": 1e-4, "80": 1e-5, "110": 1e-6}
#     learning_rate_schedule = {"0": 1e-5, "5": 1e-4, "30": 1e-5, "45": 1e-6}
    
    # read data
    train_params = {"batch_size": batch_size,
                       "shuffle": True,
                       "drop_last": True,
                   "collate_fn": custom_collate_fn}

    valid_params = {"batch_size": batch_size,
                   "shuffle": False,
                   "drop_last": False,
                   "collate_fn": custom_collate_fn}
    
    train_set = yoloDataset(root, root+'voc2007train.txt',train=True)
    train_generator = DataLoader(train_set, **train_params)

    valid_set = yoloDataset(root, root+'voc2007valid.txt',train=False)
    valid_generator = DataLoader(valid_set, **valid_params)
    
    # load pretrain weight
    if torch.cuda.is_available():
        model = Yolov2(train_set.num_classes)
#         model.load_state_dict(torch.load(pre_trained_model_path))
    else:
        model = Yolov2(train_set.num_classes)
#         model.load_state_dict(torch.load(pre_trained_model_path, map_location=lambda storage, loc: storage))
    # The following line will re-initialize weight for the last layer, which is useful
    # when you want to retrain the model based on my trained weights. if you uncomment it,
    # you will see the loss is already very small at the beginning.
#     nn.init.normal_(list(model.modules())[-1].weight, 0, 0.01)
    
    if torch.cuda.is_available():
        model.cuda()

    criterion = YoloLoss(train_set.num_classes, model.anchors, reduction)
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-5, momentum=momentum, weight_decay=decay)
    best_loss = 1e10
    best_epoch = 0
    
    # start train and valid
    num_iter_per_epoch = len(train_generator)
    for epoch in range(epoches):
        # scheduler change learning rate
        if str(epoch) in learning_rate_schedule.keys():
            for param_group in optimizer.param_groups:
                param_group['lr'] = learning_rate_schedule[str(epoch)]
                
        # batch train
        model.train()
        for iter, batch in enumerate(train_generator):
            image, label = batch
            if torch.cuda.is_available():
                image = Variable(image.cuda(), requires_grad=True)
            else:
                image = Variable(image, requires_grad=True)
            optimizer.zero_grad()
            logits = model(image)
            loss, loss_coord, loss_conf, loss_cls = criterion(logits, label)
            loss.backward()
            optimizer.step()
            print("Train Epoch: {}/{}, Iteration: {}/{}, Lr: {}, Loss:{:.2f} (Coord:{:.2f} Conf:{:.2f} Cls:{:.2f})".format(
                epoch + 1,
                epoches,
                iter + 1,
                num_iter_per_epoch,
                optimizer.param_groups[0]['lr'],
                loss,
                loss_coord,
                loss_conf,
                loss_cls))
        
        # valid
        model.eval()
        loss_ls = []
        loss_coord_ls = []
        loss_conf_ls = []
        loss_cls_ls = []
        for valid_iter, valid_batch in enumerate(valid_generator):
            valid_image, valid_label = valid_batch
            num_sample = len(valid_label)

            if torch.cuda.is_available():
                valid_image = valid_image.cuda()

            with torch.no_grad():
                valid_logits = model(valid_image)
                batch_loss, batch_loss_coord, batch_loss_conf, batch_loss_cls = criterion(valid_logits, valid_label)
            # record loss
            loss_ls.append(batch_loss * num_sample)
            loss_coord_ls.append(batch_loss_coord * num_sample)
            loss_conf_ls.append(batch_loss_conf * num_sample)
            loss_cls_ls.append(batch_loss_cls * num_sample)

        valid_loss = sum(loss_ls) / valid_set.__len__()
        valid_coord_loss = sum(loss_coord_ls) / valid_set.__len__()
        valid_conf_loss = sum(loss_conf_ls) / valid_set.__len__()
        valid_cls_loss = sum(loss_cls_ls) / valid_set.__len__()
        print("\nTest Epoch: {}/{}, Lr: {}, Loss:{:.2f} (Coord:{:.2f} Conf:{:.2f} Cls:{:.2f})\n".format(
            epoch + 1,
            epoches,
            optimizer.param_groups[0]['lr'],
            valid_loss,
            valid_coord_loss,
            valid_conf_loss,
            valid_cls_loss))
            
        # save the best model weight
        if valid_loss + es_min_delta < best_loss:
            best_loss = valid_loss
            best_epoch = epoch
            torch.save(model.state_dict(), os.path.join('model_best_v2.pth'))

        # Early stopping
        if epoch - best_epoch > epoch_patience > 0:
            print("Stop training at epoch {}. The lowest loss achieved is {}".format(epoch, valid_loss))
            break

In [61]:
train()

/tmp/ipykernel_431703/3533180086.py:36: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  lin_x = torch.range(0, width - 1).repeat(height, 1).view(height * width)
/tmp/ipykernel_431703/3533180086.py:37: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  lin_y = torch.range(0, height - 1).repeat(width, 1).t().contiguous().view(height * width)
/home/andrea/anaconda3/envs/Lab_env/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Train Epoch: 1/100, Iteration: 1/438, Lr: 1e-05, Loss:256.85 (Coord:2.98 Conf:234.36 Cls:19.51)
Train Epoch: 1/100, Iteration: 2/438, Lr: 1e-05, Loss:242.96 (Coord:2.35 Conf:226.41 Cls:14.19)
Train Epoch: 1/100, Iteration: 3/438, Lr: 1e-05, Loss:218.44 (Coord:1.49 Conf:206.28 Cls:10.67)
Train Epoch: 1/100, Iteration: 4/438, Lr: 1e-05, Loss:196.61 (Coord:2.51 Conf:181.13 Cls:12.98)
Train Epoch: 1/100, Iteration: 5/438, Lr: 1e-05, Loss:167.74 (Coord:1.78 Conf:154.43 Cls:11.52)
Train Epoch: 1/100, Iteration: 6/438, Lr: 1e-05, Loss:143.15 (Coord:2.03 Conf:124.12 Cls:17.00)
Train Epoch: 1/100, Iteration: 7/438, Lr: 1e-05, Loss:115.87 (Coord:1.80 Conf:99.77 Cls:14.30)
Train Epoch: 1/100, Iteration: 8/438, Lr: 1e-05, Loss:103.82 (Coord:3.08 Conf:74.59 Cls:26.15)
Train Epoch: 1/100, Iteration: 9/438, Lr: 1e-05, Loss:70.24 (Coord:1.51 Conf:56.30 Cls:12.43)
Train Epoch: 1/100, Iteration: 10/438, Lr: 1e-05, Loss:64.59 (Coord:2.47 Conf:40.94 Cls:21.18)
Train Epoch: 1/100, Iteration: 11/438, Lr: 1e

Train Epoch: 1/100, Iteration: 91/438, Lr: 1e-05, Loss:13.05 (Coord:1.08 Conf:2.03 Cls:9.94)
Train Epoch: 1/100, Iteration: 92/438, Lr: 1e-05, Loss:20.57 (Coord:1.67 Conf:3.88 Cls:15.02)
Train Epoch: 1/100, Iteration: 93/438, Lr: 1e-05, Loss:12.43 (Coord:1.07 Conf:2.91 Cls:8.45)
Train Epoch: 1/100, Iteration: 94/438, Lr: 1e-05, Loss:15.68 (Coord:1.22 Conf:3.16 Cls:11.30)
Train Epoch: 1/100, Iteration: 95/438, Lr: 1e-05, Loss:20.39 (Coord:2.41 Conf:3.10 Cls:14.87)
Train Epoch: 1/100, Iteration: 96/438, Lr: 1e-05, Loss:22.46 (Coord:2.64 Conf:3.27 Cls:16.56)
Train Epoch: 1/100, Iteration: 97/438, Lr: 1e-05, Loss:22.02 (Coord:1.90 Conf:3.99 Cls:16.13)
Train Epoch: 1/100, Iteration: 98/438, Lr: 1e-05, Loss:13.03 (Coord:0.79 Conf:2.99 Cls:9.24)
Train Epoch: 1/100, Iteration: 99/438, Lr: 1e-05, Loss:12.77 (Coord:1.34 Conf:2.32 Cls:9.11)
Train Epoch: 1/100, Iteration: 100/438, Lr: 1e-05, Loss:28.78 (Coord:1.91 Conf:6.15 Cls:20.71)
Train Epoch: 1/100, Iteration: 101/438, Lr: 1e-05, Loss:17.63 (

Train Epoch: 1/100, Iteration: 180/438, Lr: 1e-05, Loss:17.34 (Coord:1.43 Conf:3.96 Cls:11.95)
Train Epoch: 1/100, Iteration: 181/438, Lr: 1e-05, Loss:24.15 (Coord:2.24 Conf:4.32 Cls:17.60)
Train Epoch: 1/100, Iteration: 182/438, Lr: 1e-05, Loss:16.06 (Coord:1.84 Conf:3.18 Cls:11.04)
Train Epoch: 1/100, Iteration: 183/438, Lr: 1e-05, Loss:14.84 (Coord:1.10 Conf:2.85 Cls:10.88)
Train Epoch: 1/100, Iteration: 184/438, Lr: 1e-05, Loss:25.82 (Coord:2.19 Conf:5.38 Cls:18.26)
Train Epoch: 1/100, Iteration: 185/438, Lr: 1e-05, Loss:14.13 (Coord:1.22 Conf:3.36 Cls:9.54)
Train Epoch: 1/100, Iteration: 186/438, Lr: 1e-05, Loss:16.21 (Coord:1.50 Conf:3.41 Cls:11.30)
Train Epoch: 1/100, Iteration: 187/438, Lr: 1e-05, Loss:14.74 (Coord:1.43 Conf:2.75 Cls:10.55)
Train Epoch: 1/100, Iteration: 188/438, Lr: 1e-05, Loss:17.89 (Coord:1.37 Conf:3.60 Cls:12.91)
Train Epoch: 1/100, Iteration: 189/438, Lr: 1e-05, Loss:15.59 (Coord:1.43 Conf:2.71 Cls:11.45)
Train Epoch: 1/100, Iteration: 190/438, Lr: 1e-05, 

Train Epoch: 1/100, Iteration: 267/438, Lr: 1e-05, Loss:13.23 (Coord:1.07 Conf:2.60 Cls:9.56)
Train Epoch: 1/100, Iteration: 268/438, Lr: 1e-05, Loss:19.13 (Coord:1.30 Conf:3.47 Cls:14.36)
Train Epoch: 1/100, Iteration: 269/438, Lr: 1e-05, Loss:10.92 (Coord:0.77 Conf:2.33 Cls:7.82)
Train Epoch: 1/100, Iteration: 270/438, Lr: 1e-05, Loss:12.14 (Coord:1.32 Conf:2.23 Cls:8.59)
Train Epoch: 1/100, Iteration: 271/438, Lr: 1e-05, Loss:16.68 (Coord:1.37 Conf:2.73 Cls:12.58)
Train Epoch: 1/100, Iteration: 272/438, Lr: 1e-05, Loss:18.25 (Coord:1.56 Conf:3.52 Cls:13.17)
Train Epoch: 1/100, Iteration: 273/438, Lr: 1e-05, Loss:13.99 (Coord:1.22 Conf:2.68 Cls:10.08)
Train Epoch: 1/100, Iteration: 274/438, Lr: 1e-05, Loss:16.58 (Coord:1.60 Conf:2.94 Cls:12.04)
Train Epoch: 1/100, Iteration: 275/438, Lr: 1e-05, Loss:12.79 (Coord:0.95 Conf:3.27 Cls:8.57)
Train Epoch: 1/100, Iteration: 276/438, Lr: 1e-05, Loss:12.12 (Coord:0.92 Conf:3.07 Cls:8.13)
Train Epoch: 1/100, Iteration: 277/438, Lr: 1e-05, Loss

...

Train Epoch: 44/100, Iteration: 85/438, Lr: 0.0001, Loss:2.55 (Coord:0.66 Conf:1.73 Cls:0.16)
Train Epoch: 44/100, Iteration: 86/438, Lr: 0.0001, Loss:1.94 (Coord:0.31 Conf:1.46 Cls:0.17)
Train Epoch: 44/100, Iteration: 87/438, Lr: 0.0001, Loss:2.01 (Coord:0.45 Conf:1.52 Cls:0.05)
Train Epoch: 44/100, Iteration: 88/438, Lr: 0.0001, Loss:1.28 (Coord:0.16 Conf:1.08 Cls:0.05)
Train Epoch: 44/100, Iteration: 89/438, Lr: 0.0001, Loss:3.19 (Coord:0.45 Conf:2.42 Cls:0.32)
Train Epoch: 44/100, Iteration: 90/438, Lr: 0.0001, Loss:2.42 (Coord:0.36 Conf:1.76 Cls:0.31)
Train Epoch: 44/100, Iteration: 91/438, Lr: 0.0001, Loss:2.77 (Coord:0.68 Conf:1.97 Cls:0.12)
Train Epoch: 44/100, Iteration: 92/438, Lr: 0.0001, Loss:3.39 (Coord:0.65 Conf:2.54 Cls:0.19)
Train Epoch: 44/100, Iteration: 93/438, Lr: 0.0001, Loss:2.63 (Coord:0.32 Conf:2.09 Cls:0.22)
Train Epoch: 44/100, Iteration: 94/438, Lr: 0.0001, Loss:1.70 (Coord:0.32 Conf:1.00 Cls:0.37)
Train Epoch: 44/100, Iteration: 95/438, Lr: 0.0001, Loss:1.7

Train Epoch: 44/100, Iteration: 172/438, Lr: 0.0001, Loss:2.56 (Coord:0.40 Conf:2.07 Cls:0.09)
Train Epoch: 44/100, Iteration: 173/438, Lr: 0.0001, Loss:2.10 (Coord:0.31 Conf:1.30 Cls:0.49)
Train Epoch: 44/100, Iteration: 174/438, Lr: 0.0001, Loss:3.18 (Coord:0.29 Conf:2.67 Cls:0.22)
Train Epoch: 44/100, Iteration: 175/438, Lr: 0.0001, Loss:1.85 (Coord:0.48 Conf:1.34 Cls:0.02)
Train Epoch: 44/100, Iteration: 176/438, Lr: 0.0001, Loss:2.81 (Coord:0.34 Conf:2.25 Cls:0.21)
Train Epoch: 44/100, Iteration: 177/438, Lr: 0.0001, Loss:3.03 (Coord:0.52 Conf:1.89 Cls:0.61)
Train Epoch: 44/100, Iteration: 178/438, Lr: 0.0001, Loss:2.95 (Coord:0.32 Conf:2.48 Cls:0.15)
Train Epoch: 44/100, Iteration: 179/438, Lr: 0.0001, Loss:1.61 (Coord:0.26 Conf:0.88 Cls:0.47)
Train Epoch: 44/100, Iteration: 180/438, Lr: 0.0001, Loss:3.51 (Coord:0.41 Conf:2.88 Cls:0.21)
Train Epoch: 44/100, Iteration: 181/438, Lr: 0.0001, Loss:2.82 (Coord:0.38 Conf:2.07 Cls:0.37)
Train Epoch: 44/100, Iteration: 182/438, Lr: 0.000

Train Epoch: 44/100, Iteration: 260/438, Lr: 0.0001, Loss:3.57 (Coord:0.51 Conf:2.76 Cls:0.30)
Train Epoch: 44/100, Iteration: 261/438, Lr: 0.0001, Loss:2.26 (Coord:0.49 Conf:1.66 Cls:0.12)
Train Epoch: 44/100, Iteration: 262/438, Lr: 0.0001, Loss:3.15 (Coord:0.35 Conf:2.13 Cls:0.67)
Train Epoch: 44/100, Iteration: 263/438, Lr: 0.0001, Loss:1.74 (Coord:0.24 Conf:1.22 Cls:0.28)
Train Epoch: 44/100, Iteration: 264/438, Lr: 0.0001, Loss:5.21 (Coord:0.82 Conf:4.15 Cls:0.23)
Train Epoch: 44/100, Iteration: 265/438, Lr: 0.0001, Loss:1.74 (Coord:0.30 Conf:1.39 Cls:0.05)
Train Epoch: 44/100, Iteration: 266/438, Lr: 0.0001, Loss:2.95 (Coord:0.54 Conf:2.29 Cls:0.12)
Train Epoch: 44/100, Iteration: 267/438, Lr: 0.0001, Loss:2.45 (Coord:0.33 Conf:1.77 Cls:0.35)
Train Epoch: 44/100, Iteration: 268/438, Lr: 0.0001, Loss:2.47 (Coord:0.27 Conf:2.14 Cls:0.06)
Train Epoch: 44/100, Iteration: 269/438, Lr: 0.0001, Loss:2.04 (Coord:0.61 Conf:1.29 Cls:0.14)
Train Epoch: 44/100, Iteration: 270/438, Lr: 0.000

Train Epoch: 44/100, Iteration: 348/438, Lr: 0.0001, Loss:1.93 (Coord:0.33 Conf:1.53 Cls:0.07)
Train Epoch: 44/100, Iteration: 349/438, Lr: 0.0001, Loss:2.22 (Coord:0.22 Conf:1.83 Cls:0.17)
Train Epoch: 44/100, Iteration: 350/438, Lr: 0.0001, Loss:3.18 (Coord:0.32 Conf:2.56 Cls:0.31)
Train Epoch: 44/100, Iteration: 351/438, Lr: 0.0001, Loss:3.99 (Coord:0.32 Conf:3.47 Cls:0.20)
Train Epoch: 44/100, Iteration: 352/438, Lr: 0.0001, Loss:2.54 (Coord:0.48 Conf:1.79 Cls:0.27)
Train Epoch: 44/100, Iteration: 353/438, Lr: 0.0001, Loss:1.78 (Coord:0.29 Conf:1.45 Cls:0.04)
Train Epoch: 44/100, Iteration: 354/438, Lr: 0.0001, Loss:2.47 (Coord:0.50 Conf:1.91 Cls:0.06)
Train Epoch: 44/100, Iteration: 355/438, Lr: 0.0001, Loss:2.64 (Coord:0.36 Conf:2.09 Cls:0.19)
Train Epoch: 44/100, Iteration: 356/438, Lr: 0.0001, Loss:2.68 (Coord:0.30 Conf:2.29 Cls:0.10)
Train Epoch: 44/100, Iteration: 357/438, Lr: 0.0001, Loss:3.33 (Coord:0.49 Conf:2.74 Cls:0.11)
Train Epoch: 44/100, Iteration: 358/438, Lr: 0.000

Train Epoch: 44/100, Iteration: 435/438, Lr: 0.0001, Loss:3.16 (Coord:0.38 Conf:2.39 Cls:0.39)
Train Epoch: 44/100, Iteration: 436/438, Lr: 0.0001, Loss:1.49 (Coord:0.21 Conf:1.26 Cls:0.02)
Train Epoch: 44/100, Iteration: 437/438, Lr: 0.0001, Loss:1.91 (Coord:0.33 Conf:1.53 Cls:0.05)
Train Epoch: 44/100, Iteration: 438/438, Lr: 0.0001, Loss:3.07 (Coord:0.48 Conf:2.31 Cls:0.27)

Test Epoch: 44/100, Lr: 0.0001, Loss:11.59 (Coord:0.62 Conf:4.02 Cls:6.94)

Stop training at epoch 43. The lowest loss achieved is 11.59229850769043


In [ ]:
15.66 (Coord:1.30 Conf:3.28 Cls:11.08)
15.38 (Coord:0.77 Conf:3.91 Cls:10.70)